# 三维地图柱状图可视化
## 请选择年份
年份的可选值： 2010, 2011, 2012, 2014, 2016, 2018

In [1]:
year = 2010

## 展示table

In [2]:
print([table for table in cfps[year]])

['adult', 'child', 'comm', 'famconf', 'famecon']


### 选择一个table

In [3]:
table = 'child'

In [4]:
schema = cfps[year][table].schema

## 选择其中一列数据，
这里可选变量与您所选择的table、year有关

In [5]:
print("您可选择的变量有:")
for var,value in schema.items():
    if value['type'] in ['int32','float64']: 
        print(f'{var}:{value["key"]}')

您可选择的变量有:
pid:个人id
fid:家庭样本代码
cid:村居样本代码
countyid:区县顺序码
psu:primary sampling unit
rswt_nat:个人权重-全国样本
rswt_res:个人权重-全国再抽样样本
indno:个人的家庭成员顺序号
cyear:采访年份
cmonth:采访月份
wa1y:请问孩子的出生日期（年）
wa1m:请问孩子的出生日期（月）
wa1age:孩子的年龄（年）
wa1month:孩子的年龄（月）
wa101:孩子的胎龄（月）
wa102:孩子出生时的体重（斤）
wa103:孩子现在的体重（斤）
wa104:孩子现在的身高（厘米）
wa105:到现在为止，孩子吃母乳有多长时间（月）
wa107c_code:孩子的出生地（县/区编码）
wa501c_code:孩子现在的户口子所在地（县/区编码）
wb201:孩子父母最近非假期的1个月，孩子平均每周能见到父母几次
wb501:孩子几个月大的时候开始自己走路
wc1:上个月孩子生病了几次
wc101:上个月孩子因病去医院看病几次
wc4:去年，孩子因病去医院看病几次
wc7:去年，因孩子患病，一共花了多少（元）
wc701:孩子患病的所有花费中，家里支付了多少（元）
wc802:去年，家里为孩子另外购买商业医疗保险花费了多少（元）
wb4:去年，父母双方都不与孩子在一起居住的连续时间最长为(周)
wb601:孩子几个月大的时候开始说完整的句子，如“我要吃饭”
wb701:孩子几个月大的时候开始能数1-10的
wb801:孩子几个月大的时候开始能独立小便
wc2:孩子在1岁以前共生病几次
wc201:孩子在1岁以前，因病去医院/医疗场所看了几次病
wd501:去年全年，这个孩子的所有教育支出（元）：学杂费
wd502:去年全年，这个孩子的所有教育支出（元）：书本费
wd503:去年全年，这个孩子的所有教育支出（元）：课外辅导/家教费
wd504:去年全年，这个孩子的所有教育支出（元）：住宿费
wd505:去年全年，这个孩子的所有教育支出（元）：交通费
wd506:去年全年，这个孩子的所有教育支出（元）：其他费用
wd5total:去年全年，这个孩子的所有教育支出总计（元）
wd510:去年全年，这个孩子的所有教育支出中，您家庭支付了多少（元）
wg101:最近

### 提取location列和关心的数据列

In [6]:
loca = 'provcd'
var = 'wm302'

### 查看提取列的基本信息

In [7]:
info = cfps[year][table].schema[var]
lc = cfps[year][table].schema[loca]
info,lc

({'type': 'int32',
  'key': '你觉得自己有多幸福',
  'range': [-8, 5, 4, 1, 3, 2, -2, -1],
  'details': [],
  'minmax': [-8.0, 5.0]},
 {'type': 'enum',
  'key': '省国标码',
  'range': {'-10': '无法判断',
   '-9': '缺失',
   '-8': '不适用',
   '-2': '拒绝回答',
   '-1': '不知道',
   '11': '北京市',
   '12': '天津市',
   '13': '河北省',
   '14': '山西省',
   '15': '内蒙古自治区',
   '21': '辽宁省',
   '22': '吉林省',
   '23': '黑龙江省',
   '31': '上海市',
   '32': '江苏省',
   '33': '浙江省',
   '34': '安徽省',
   '35': '福建省',
   '36': '江西省',
   '37': '山东省',
   '41': '河南省',
   '42': '湖北省',
   '43': '湖南省',
   '44': '广东省',
   '45': '广西壮族自治区',
   '46': '海南省',
   '50': '重庆市',
   '51': '四川省',
   '52': '贵州省',
   '53': '云南省',
   '54': '西藏自治区',
   '61': '陕西省',
   '62': '甘肃省',
   '63': '青海省',
   '64': '宁夏回族自治区',
   '65': '新疆维吾尔自治区'},
  'details': [],
  'minmax': [11.0, 62.0]})

#### 数据处理

In [8]:
condi = f'{var}>0 and {loca}>0'

In [9]:
df = sql(f'select {loca},{var} from {table}_{year} where {condi}')

In [10]:
# 各省对应变量数据加和取平均
data = df.groupby(loca,as_index=False).mean()

In [56]:
title = info['key']
axis = {}
for i in data[loca]:
    name = lc['range'][str(i)]
    index = np.argwhere(data[loca].values==i)[0][0]
    axis[name] = round(data[var][index],2)

In [57]:
latlon_data = {
    '山东省':[117.000923, 36.675807],
    '河北省':[115.48333,38.03333],
    '吉林省':[125.35000,43.88333],
    '黑龙江省':[127.63333,47.75000],
    '辽宁省':[123.38333,41.80000],
    '内蒙古自治区':[111.670801, 41.818311],
    '新疆维吾尔自治区':[87.68333,43.76667],
    '甘肃省':[103.73333,36.03333],
    '宁夏回族自治区':[106.26667,37.46667],
    '山西省':[112.53333,37.86667],
    '陕西省':[108.95000,34.26667],
    '河南省':[113.65000,34.76667],
    '安徽省':[117.283042, 31.86119],
    '江苏省':[119.78333,32.05000],
    '浙江省':[120.20000,30.26667],
    '福建省':[118.30000,26.08333],
    '广东省':[113.23333,23.16667],
    '江西省':[115.90000,28.68333],
    '海南省':[110.35000,20.01667],
    '广西壮族自治区':[108.320004, 22.82402],
    '贵州省':[106.71667,26.56667],
    '湖南省':[113.00000,28.21667],
    '湖北省':[114.298572, 30.584355],
    '四川省':[104.06667,30.66667],
    '云南省':[102.73333,25.05000],
    '西藏自治区':[91.00000,30.60000],
    '青海省':[96.75000,36.56667],
    '天津市':[117.20000,39.13333],
    '上海市':[121.55333,31.20000],
    '重庆市':[106.45000, 29.56667],
    '北京市': [116.41667,39.91667],
    '台湾省': [121.30, 25.03],
}
to_be_deleted = []
for key, value in latlon_data.items():
    if axis.get(key):
        value.append(axis[key])
    else:
        to_be_deleted.append(key)
for key in to_be_deleted:
    del latlon_data[key]

In [60]:
(
    Map3D()
    .add_schema(
        itemstyle_opts=opts.ItemStyleOpts(
            color="#20b2aa",
            opacity=1,
            border_width=0.8,
            border_color="cyan",
        ),
        map3d_label=opts.Map3DLabelOpts(
            is_show=False,
            formatter=JsCode("function(data){return data.name + " " + data.value[2];}"),
        ),
        emphasis_label_opts=opts.LabelOpts(
            is_show=False,
            color="#fff",
            font_size=10,
            background_color="rgba(0,23,11,0)",
        ),
        light_opts=opts.Map3DLightOpts(
            main_color="#fff",
            main_intensity=1.2,
            main_shadow_quality="high",
            is_main_shadow=False,
            main_beta=10,
            ambient_intensity=0.3,
        ),
    )
    .add(
        series_name=info['key'],
        data_pair=list(latlon_data.items()),
        type_=ChartType.BAR3D,
        bar_size=1,
        shading="lambert",
        label_opts=opts.LabelOpts(
            is_show=False,
            formatter=JsCode("function(data){return data.name + ' ' + data.value[2];}"),
        )
    )
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False), type_=ChartType.BAR3D, color=['hotpink'])
    .set_global_opts(title_opts=opts.TitleOpts(title=title))
    .render_notebook()
)